# Mandatory Exercise - Session 7

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this lab is toread all the pairs of sentences, compute their similarities using words + Nammed Enntities (NEs) and Jaccard Coefficient and show the results. 

In [1]:
# initial imports. Could also be done in the PC
import nltk
import nltk.corpus
nltk.download('maxent_ne_chunker')
nltk.download('words')

#additional set of imports
from nltk import pos_tag
from nltk.metrics import jaccard_distance
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.parse import CoreNLPParser

# Core Named-entity tagger as stanford one is deprecated: https://github.com/nltk/nltk/issues/2010
tagger = CoreNLPParser(url='http://localhost:9000', tagtype='ner')

# launch the server in terminal
# java -mx4g -cp "/home/jan/Downloads/stanford-corenlp-full-2018-10-05/*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/jan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/jan/nltk_data...
[nltk_data]   Package words is already up-to-date!


### 1.

In [2]:
def get_stanford_named_entity_chunked(sentence):
    """Given the passed sentence string, returns an array with the chunks (words and named entities) it contains, using Stanford NLP"""
        
    # obtain an array with the sentence tokens
    tokenized_s = nltk.word_tokenize(sentence)
    
    # tag and run as a normal word or a named entity (e.g. a person or an organization)
    tagged_s = tagger.tag(tokenized_s)
    
    chunked_sentence = []
    last_token = ''
    last_tag = ''
    
    for tagged_token in tagged_s:
        
        token = tagged_token[0]
        tag = tagged_token[1]
        
        # make normal words have lower case, also discard punctuation marks
        if tag == 'O':
            if token.isalnum():
                chunked_sentence.append(token.lower())
         
        # keep named entities with the original capitalization
        else:
            if last_tag == tag:
                chunked_sentence[-1] += ' ' + token
            else:
                chunked_sentence.append(token)
        
        last_token = token
        last_tag = tag
    
    return chunked_sentence

# example: note it does not group the terms of named entities, always 1 by 1. 
print(get_stanford_named_entity_chunked("Mark Pedersen and John Smith are working at Google since 1994 for 1000$ per week."))

['Mark Pedersen', 'and', 'John Smith', 'are', 'working', 'at', 'Google', 'since', '1994', 'for', '1000', '$', 'per', 'week']


### 2. 

In [3]:
def get_ne_chunked(sentence):
    """Given the passed sentence string, returns an array with the chunks (words and named entities) it contains, using NLTK"""
        
    # obtain an array with the sentence tokens
    tokenized_s = nltk.word_tokenize(sentence)
        
    # chunk and tag with NLTK named-entity tagged
    x = pos_tag(word_tokenize(sentence))
    chunk_tree = ne_chunk(x, binary=True)
    
    chunked_sentence = []
    for chunk in chunk_tree:
        
        # keep named entities with the original capitalization
        if hasattr(chunk, 'label'):
            token = ' '.join(term[0] for term in chunk)
            chunked_sentence.append(token)
            
        # make normal words have lower case, also discard puntuaction marks
        else:
            token = chunk[0]
            if token.isalnum():
                chunked_sentence.append(token.lower())
    
    return chunked_sentence

# example: note that NLTK naming-entity tagger does group the terms of named entities in a single string
print(get_ne_chunked("Mark Pedersen and John Smith are working at Google since 1994 for 1000$ per week."))

['Mark Pedersen', 'and', 'John Smith', 'are', 'working', 'at', 'Google', 'since', '1994', 'for', '1000', 'per', 'week']


We now proceed to open and read the input file

In [10]:
import os
import sys

# full path of the corpus file, with that the trial folder containing the input file being in the same directory as the JPN
absolute_file_path = os.path.dirname(os.path.abspath("__file__")) + "//trial//STS.input.txt"

#value initialization and instantiation
d = {}
tests = []
standard = []

# find all sentence pairs in the document
sentence_pairs = []
sentence_set_pairs = []
with open(absolute_file_path) as f:
    lines = f.readlines()
    for line in lines:
        index, sentence0, sentence1 = line.split("\t")
        if index in d:
            d[index] = sentence0, sentence1
        else:
            d[index] = (get_stanford_named_entity_chunked(sentence0), get_stanford_named_entity_chunked(sentence1))
            print("First sentence: \t", sentence0, "\nSecond sentence: \t", sentence1, "\n")
    print()  

First sentence: 	 The bird is bathing in the sink. 
Second sentence: 	 Birdie is washing itself in the water basin.
 

First sentence: 	 In May 2010, the troops attempted to invade Kabul. 
Second sentence: 	 The US army invaded Kabul on May 7th last year, 2010.
 

First sentence: 	 John said he is considered a witness but not a suspect. 
Second sentence: 	 "He is not a suspect anymore." John said.
 

First sentence: 	 They flew out of the nest in groups. 
Second sentence: 	 They flew into the nest together.
 

First sentence: 	 The woman is playing the violin. 
Second sentence: 	 The young lady enjoys listening to the guitar.
 

First sentence: 	 John went horse back riding at dawn with a whole group of friends. 
Second sentence: 	 Sunrise at dawn is a magnificent view to take in if you wake up early enough for it.
 




### 2. Sentence similarity calculation using words and named entities, compared with the gold standard.
The pairs of sentences are checked to see how similar they are, using the Jaccard distance: the more words or named entities two sentences share, the more alike they are considered to be. The pairs are shown along with their distance and dissimilarity score (scaled to be comparable with the gold standard one).

In [5]:
from nltk.metrics import jaccard_distance
from scipy.stats import pearsonr

# gold standard file path
absolute_file_path = os.path.dirname(os.path.abspath("__file__")) + "//trial//STS.gs.txt"

# get the gold standard scores
gold_scores = []
with open(absolute_file_path) as f:
    lines = f.readlines()
    for line in lines:
        _, score = line.split("\t")
        gold_scores.append(int(score))
        
word_ne_scores = []

# compute the Jaccard distance to see how similar or different two sentences are
for i in range(len(sentence_pairs)):
    pair = sentence_pairs[i]
    word_ne_dist = jaccard_distance(set(pair[0]), set(pair[1]))
    word_ne_score = round(word_ne_dist * 5)
    word_ne_scores.append(word_ne_score)
    print("First sentence words and named entities: ", pair[0], "\nSecond sentence words and named entities: ", pair[1], "\nWord-and-named-entity-based distance:", round(word_ne_dist, 3), "\nWord-and-named-entity-based dissimilarity score:", word_ne_score, "\nGold standard dissimilarity score:", gold_scores[i], "\n") 

# Pearson correlation between the tested and the gold standard scores
word_ne_pearson = pearsonr(word_ne_scores, gold_scores)
print("Pearson correlation between word-and-named-entity-based method and gold standard:", round(word_ne_pearson[0], 3))

First sentence words and named entities:  ['the', 'bird', 'is', 'bathing', 'in', 'the', 'sink'] 
Second sentence words and named entities:  ['birdie', 'is', 'washing', 'itself', 'in', 'the', 'water', 'basin'] 
Word-and-named-entity-based distance: 0.727 
Word-and-named-entity-based dissimilarity score: 4 
Gold standard dissimilarity score: 5 

First sentence words and named entities:  ['in', 'May 2010', 'the', 'troops', 'attempted', 'to', 'invade', 'Kabul'] 
Second sentence words and named entities:  ['the', 'US', 'army', 'invaded', 'Kabul', 'on', 'May 7th last year , 2010'] 
Word-and-named-entity-based distance: 0.846 
Word-and-named-entity-based dissimilarity score: 4 
Gold standard dissimilarity score: 4 

First sentence words and named entities:  ['John', 'said', 'he', 'is', 'considered', 'a', 'witness', 'but', 'not', 'a', 'suspect'] 
Second sentence words and named entities:  ['he', 'is', 'not', 'a', 'suspect', 'anymore', 'John', 'said'] 
Word-and-named-entity-based distance: 0.36